# Assignment 3 ( Question 4) Know the hatred!

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- all the packages that are imported in the template code

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q4.ipynb`.


In [1]:
!pip install scikit-multilearn

     |████████████████████████████████| 89 kB 5.0 MB/s 


In [2]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import re
import sklearn
import torch
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
!pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModel

     |████████████████████████████████| 4.0 MB 10.8 MB/s 
     |████████████████████████████████| 596 kB 12.9 MB/s 
     |████████████████████████████████| 6.6 MB 44.9 MB/s 
     |████████████████████████████████| 895 kB 41.3 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#Dataset

In the training data, the comments are labelled as one or more of the six categories; toxic, severe toxic, obscene, threat, insult and identity hate. This is essentially a multi-label classification problem.

The dataset here is a multi-label classification dataset. To understand multi-label classification datasets, you can refer here: https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/ 

.


Here, we will be using Binary relevance to solve our multi-label classification problem



**Binary Relevance**: This is probably the simplest which treats each label as a separate single classification problems. The key assumption here though, is that there are no correlation among the various labels.

Finally, we will be summing up the accuracies obtained of all labels to obtain the final accuracy


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# loading the dataset

# train_df = pd.read_csv('q4_data/train.csv')
# test_df = pd.read_csv('q4_data/test.csv')
train_df = pd.read_csv('/content/gdrive/MyDrive/SMAI/Assignment 3/train.csv')
test_df = pd.read_csv('/content/gdrive/MyDrive/SMAI/Assignment 3/test.csv')
test_labels_df = pd.read_csv('/content/gdrive/MyDrive/SMAI/Assignment 3/test_labels.csv')

#dataset understanding
train_df.sample(5)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
146571,2e383a1f2c041620,]] | [[User talk:Huwmanbeing/2014|2014,0,0,0,0,0,0
90230,f17585028582c396,"In you read the sentences, Eric, you would see...",0,0,0,0,0,0
5853,0fa4821f1b5636e1,REDIRECT Talk:Chucky (Child's Play),0,0,0,0,0,0
30796,51cee396a2cde319,"""\n\n Consonant and Vowel inventory?? \n\nHey ...",0,0,0,0,0,0
3391,092541ac794816e3,I added a link to the full original text and s...,0,0,0,0,0,0


In [6]:
test_df.shape

(153164, 2)

In [7]:
test_df.drop(test_labels_df.index[(test_labels_df['toxic'] ==-1) | (test_labels_df['severe_toxic'] ==-1) | (test_labels_df['obscene'] ==-1) | (test_labels_df['threat'] ==-1) | (test_labels_df['insult'] ==-1) | (test_labels_df['identity_hate'] ==-1)], axis=0,inplace=True)

In [8]:
test_labels_df.drop(test_labels_df.index[(test_labels_df['toxic'] ==-1) | (test_labels_df['severe_toxic'] ==-1) | (test_labels_df['obscene'] ==-1) | (test_labels_df['threat'] ==-1) | (test_labels_df['insult'] ==-1) | (test_labels_df['identity_hate'] ==-1)], axis=0,inplace=True)

In [9]:
test_labels_df.shape

(63978, 7)

In [10]:
test_labels_df

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,0,0,0,0,0,0
7,000247e83dcc1211,0,0,0,0,0,0
11,0002f87b16116a7f,0,0,0,0,0,0
13,0003e1cccfd5a40a,0,0,0,0,0,0
14,00059ace3e3e9a53,0,0,0,0,0,0
...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,0,0,0,0,0,0
153151,fff9d70fe0722906,0,0,0,0,0,0
153154,fffa8a11c4378854,0,0,0,0,0,0
153155,fffac2a094c8e0e2,1,0,1,0,1,0


In [11]:
# let's see the total rows in train, test data and the numbers for the various categories
cols_target = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult' , 'identity_hate']
print('Total rows in test is {}'.format(len(test_df)))
print('Total rows in train is {}'.format(len(train_df)))
print(train_df[cols_target].sum())


Total rows in test is 63978
Total rows in train is 159571
toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64


In [12]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


In [13]:
train_df['comment_text'] = train_df['comment_text'].map(lambda com : clean_text(com))
test_df['comment_text'] = test_df['comment_text'].map(lambda com : clean_text(com))

In [14]:
# train_df = train_df.drop('char_length',axis=1)
X = train_df.comment_text
test_X = test_df.comment_text
print(X.shape, test_X.shape)


(159571,) (63978,)


Now, we will be experimenting this classification problem with different classifiers. We will be using pre-trained models inorder to obtain the required embeddings. Here, we will be using BERT and Universal Sentence Encoder embeddings.


Understand how Elmo and USE(Universal Sentence Encoder) works here. You can use any of both to train your classifiers.

To understand how different sentence embeddings work, you can refer to this link: https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial 

Please give it a look before you start working with transfer learning/ sentence embeddings like BERT, Elmo, Ulm-fit, USE.


In [15]:
Y_train = train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult' , 'identity_hate']]

In [16]:
Y_train = Y_train.to_numpy()

In [17]:
Y_test = test_labels_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult' , 'identity_hate']].to_numpy()

In [18]:
#loading universal sentence encoder

import tensorflow_hub as hub
# Load pre-trained universal sentence encoder model
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# elmo_embed = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)
elmo_embed = hub.load("https://tfhub.dev/google/elmo/3")

In [19]:
def get_use_embedding(sentence):
  embedding = use_embed([sentence])
  embedding = embedding.numpy()
  return embedding[0]

In [20]:
#to obtain bert embeddings ( you need not completely understand how it works)
#you can directly use the function get_bert_embedding to obtain the embedding


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')


def get_bert_embedding(sentence):
  # encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
  encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt',max_length=512)
  with torch.no_grad():
    model_output = model(**encoded_input)
  sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentence_embedding[0]


Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
# X_train = X[0:1000]
# Y_train_temp = Y_train[0:1000]

In [21]:
X_train = X.apply(lambda sent:get_use_embedding(sent))
# X_train = X_train.apply(lambda sent:get_use_embedding(sent))

In [ ]:
#Obtain X_test and X_train 
X_test = test_X.map(lambda sent:get_use_embedding(sent))

In [ ]:
xtrain=[]
for i in X_train:
  xtrain.append(i)

In [ ]:
xtest=[]
for i in X_test:
  xtest.append(i)

In [ ]:

# from sklearn.model_selection import train_test_split
# train_X,test_X,train_Y,test_Y = train_test_split(X_train,Y_train,test_size=0.2,random_state=42)

# Classifiers
Here, we use different classifiers for our classification problem. You can directly import classifiers for first two.


*   Naive Bayes classifier
*   Support Vector Machine
*   Multi-layer Perceptron(MLP)

### Naive Bayes classifier

In [ ]:
#calculate the accuracies obtained, u can use sklearn metrics


In [ ]:
# Naive Bayes classifier 
NB_classifier = BinaryRelevance(GaussianNB())
NB_classifier.fit(xtrain, Y_train)

BinaryRelevance(classifier=GaussianNB(), require_dense=[True, True])

In [ ]:
predictions = NB_classifier.predict(xtest)
accuracy_score(Y_test,predictions)

0.7983838194379318

### Support Vector Machine

In [ ]:
#Support Vector Machines
svm_classifier = BinaryRelevance(LinearSVC())
svm_classifier.fit(xtrain, Y_train)

BinaryRelevance(classifier=LinearSVC(), require_dense=[True, True])

In [ ]:
predictions = svm_classifier.predict(xtest)
accuracy_score(Y_test,predictions)

0.8930257275938603

### Multi-layer Perceptron(MLP)

In [ ]:
#report accuracies both on train dataset and test dataset

MLP_classifier = BinaryRelevance(MLPClassifier(hidden_layer_sizes=(10,),activation='relu',solver='sgd',alpha=0,learning_rate_init=0.01,max_iter=1))
MLP_classifier.fit(xtrain, Y_train)


In [ ]:
predictions = MLP_classifier.predict(xtest)
accuracy_score(Y_test,predictions)

Which classifier of the three performed better, and why do you think so ?

Ans:

Linear SVM performed better with higher accuracy. It uses less memory because they use a subset of training points in the decision phase. SVM works well with a clear margin of separation and with high dimensional space.

#Multi-layer Perceptron

For Multi-layer Perceptron, first you will be implementing your own articial Neural network, and then compare it with sklearn's MLP layer.


Implement your own artificial Neural Network (MLP)
Few Steps to be followed(for your reference):

*   Initialise the weights randomly
*   Decide upon the number of layers you wanna have and number of neurons associated in each layer. ( Have a clear idea of what will be the size of the weight matrices).
*   Write the code for forward and backward propogations
*   Decide upon the loss function
*   Train the model
*   Predict the labels after training the model




In [ ]:
# you can remove returns statements if u think they are not required
# you can also add functions if needed

def initialise_weights():
  w1 = np.random.randn(10,512) - 0.5
  b1 = np.random.randn(10,1) - 0.5
  w2 = np.random.randn(2,10) - 0.5
  b2 = np.random.randn(2,1) - 0.5
  return w1, b1, w2, b2
  
def ReLU(x):
  return np.maximum(x,0)

def deriv_ReLU(x):
  return x>0

def softmax(x):
  z = np.exp(x)/sum(np.exp(x))
  return z


def one_hot(Y):
    # one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y = np.zeros((1,2))
    # one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y[0, Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y


def loss_function():
  return 


def forward_layer(w1, b1, w2, b2, X, activation=ReLU):
  z1 = w1.dot(X) + b1
  A1 = activation(z1)
  z2 = w2.dot(A1) + b2
  A2 = softmax(z2)
  # print("X shape",X.shape)
  # print("w1 shape",w1.shape)
  # print("w2 shape",w2.shape)
  # print("z1 shape",z1.shape)
  # print("A1 shape",A1.shape)
  # print("z2 shape",z2.shape)
  # print("A2 shape",A2.shape)
  return z1, A1, z2, A2


def backward_propogation(z1, A1, z2, A2, w1, w2, X, Y):
  m =  Y.size
  one_hot_Y = one_hot(Y)
  # print("A2 shape",A2.shape)
  dz2 = A2 - one_hot_Y
  # print("dz2 shape",dz2.shape)
  dw2 = 1/m * dz2.dot(A1.T)
  db2 = 1/m * np.sum(dz2)
  dz1 = w2.T.dot(dz2) * deriv_ReLU(z1)
  # print("dz1 shape",dz1.shape)
  dw1 = 1 / m * dz1.dot(X.T)
  db1 = 1 / m * np.sum(dz1)
  return dw1, db1, dw2, db2

def update_params(w1, w2, b1, b2, dw1, dw2, db1, db2, alpha):
  w1 = w1 - alpha*dw1
  w2 = w2 - alpha*dw2
  b1 = b1 - alpha*db1
  b2 = b2 - alpha*db2
  return w1, w2, b1, b2

def calc_predictions(x):
  # print(x)
  # print(np.argmax(x,0))
  return np.argmax(x,0)

def calc_accuracy(predictions, Y):
  return np.sum(predictions == Y)/Y.size



In [ ]:
def train(X, Y, iterations, label,alpha=0.01):
  w1, b1, w2, b2 = initialise_weights()
  for i in range(iterations):
    z1, A1, z2, A2 = forward_layer(w1, b1, w2, b2, X[i].reshape(512,1), activation=ReLU)
    dw1, db1, dw2, db2 = backward_propogation(z1, A1, z2, A2, w1, w2, X[i].reshape(512,1), Y[i][label])
    w1, w2, b1, b2 = update_params(w1, w2, b1, b2, dw1, dw2, db1, db2, alpha)
    if(i%1000 == 0):
    # print("Itertaion:",i,"Accuracy:",calc_accuracy(calc_predictions(A2),Y[i][0]))
      # print("Label",label,"Iteration",i)
  return w1, b1, w2, b2

In [ ]:
# w1, b1, w2, b2 = train(xtrain, Y_train, len(xtrain), 0,alpha=0.01)

In [ ]:
W1=[]
B1=[]
W2=[]
B2=[]
for label in range(6):
  w1, b1, w2, b2 = train(xtrain, Y_train, len(xtrain), label, alpha=0.01)
  W1.append(w1)
  B1.append(b1)
  W2.append(w2)
  B2.append(b2)
  # print("Class ",label,"Training completed")


In [ ]:
#report accuracies
#here, you need to predict for each of the label as we have trained classifiers in such a way

def predict(X, iterations, w1, b1, w2, b2):
  Y=[]
  for i in range(iterations):
    z1, A1, z2, A2 = forward_layer(w1, b1, w2, b2, X[i].reshape(512,1), activation=ReLU)
    Y.extend(calc_predictions(A2))
    # print("Iteration",i)
  return Y


In [ ]:
def predict_test(X, iterations):
  predictions=[]
  for i in range(iterations):
    Y=[]
    for j in range(6):
      z1, A1, z2, A2 = forward_layer(W1[j], B1[j], W2[j], B2[j], X[i].reshape(512,1), activation=ReLU)
      Y.extend(calc_predictions(A2))
    # print("Iteration",i)
    predictions.append(Y)
  return predictions


In [ ]:
Y_predict = predict(xtest,len(xtest), W1[0], B1[0], W2[0], B2[0])

In [ ]:
Y_predict = predict_test(xtest, len(xtest))

In [ ]:
accuracy_score(Y_test,Y_predict)

0.8995279627371909

Now, compare your implemented MLP with the sklearn MLP layer.

Which MLP performed better? 
Understand and Analyse.

MLP from scratch gave better results because the gradient descent used here was stochastic. Therefore the weights are updated after each data sample travelling through the network